# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16062957568953322278
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357302272
locality {
  bus_id: 1
}
incarnation: 12496100525209309083
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # Create the tokenizer
    tokenizer = Tokenizer()
    # Assign the tokens
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    padded = pad_sequences(x, maxlen=length, padding='post')
    return padded
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [12]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Build the layers
    inputs = Input(shape=input_shape[1:]) # Needs to return an iterable, even if dim 1
    # We need the full output of the hidden layer to pass along
    hidden = GRU(128, activation='tanh', return_sequences=True)(inputs)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(hidden)
    
    # define a learning rate (hyperparameter)
    learning_rate = 0.005
    
    # Build the model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 7s 68us/step - loss: 2.1451 - acc: 0.5256 - val_loss: nan - val_acc: 0.6048
Epoch 2/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.3889 - acc: 0.6242 - val_loss: nan - val_acc: 0.6414
Epoch 3/10
110288/110288 [==============================] - 7s 65us/step - loss: 1.2079 - acc: 0.6518 - val_loss: nan - val_acc: 0.6571
Epoch 4/10
110288/110288 [==============================] - 7s 66us/step - loss: 1.1193 - acc: 0.6605 - val_loss: nan - val_acc: 0.6604
Epoch 5/10
110288/110288 [==============================] - 7s 66us/step - loss: 1.0621 - acc: 0.6681 - val_loss: nan - val_acc: 0.6699
Epoch 6/10
110288/110288 [==============================] - 7s 65us/step - loss: 1.0149 - acc: 0.6761 - val_loss: nan - val_acc: 0.6811
Epoch 7/10
110288/110288 [==============================] - 7s 64us/step - loss: 0.9786 - acc: 0.6842 - val_loss: nan - val_acc: 0.68

## Analysis of Results

The basic model was the same for all the different tests being done. The learning rate has been changed from 0.001 to 0.1 in a factor of 10 increments to observe the sensitivity of the model to the different values of learning rates. The findings are shown below:

1. Learning rate = 0.001 - Accuracy: 48%. Translation does not make sense and repeats 'est' several times, probably as being one of the most used words
2. Learning rate = 0.01 - Accuracy: 61%. Translation is not too bad although once starts missing the words, it kinds of starts repeating itself ("new jersey est parfois parfois en mois et il est il en en"). There is definetely some more variation in the word choices, but still not good.
3. Learning rate = 0.1 - Accuracy: 61%. Translation seems about the same as the previous one, although the words choice is different in this case. Recurrent and repeated words are about the same, the most frequent ("new jersey est parfois parfois en printemps et il il est en en"). 

This last one seems to make more sense, but still, we have rerun and kept lr = 0.01 for a couple of reasons. The first one, in general terms a large learning rate is more prone to jump from one solution to another, missing local minima, and the second one is that there is no gain from making it bigger, even if the occassional translation seems better, using accuracy as the real measurement in this case.

### Update after review
Updating the parameters to the suggestion from the reviewer (lower the learning rate at the same time than incresing the GRU units from output_sequence_length to 128) improved the results to an **accuracy of 70% with a translation:** *new jersey est parfois chaud en mois et il et il est en en* that is indeed better than before

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [16]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement, by following a similar process as the previous one
    # Build the layers
    inputs = Input(shape=input_shape[1:])
    # The total size of the embeddings space is that of the vocab size, the output is equal
    # to the sentence we are processing and same dim as the next layer
    embeds = Embedding(input_dim=english_vocab_size,
                       output_dim=256)(inputs)
    # The rest of the model is similar, just we operate with the embeddings now
    hidden = GRU(output_sequence_length, activation='tanh', return_sequences=True)(embeds)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(hidden)
    
    # define a learning rate (hyperparameter)
    learning_rate = 0.001
    
    # Build the model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)


# Reshape the input, now the embeds have a different dimension as we can feed the embeddings and
# not word by word
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
simple_embed_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_embed_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=25, validation_split=0.2)

# Show a summary before testing prediction
simple_embed_model.summary()

# Print prediction(s)
print("Translate '{}': \n".format(english_sentences[0]))
print(logits_to_text(simple_embed_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/25
110288/110288 [==============================] - 8s 75us/step - loss: 4.3184 - acc: 0.4085 - val_loss: nan - val_acc: 0.4179
Epoch 2/25
110288/110288 [==============================] - 8s 69us/step - loss: 2.9779 - acc: 0.4269 - val_loss: nan - val_acc: 0.4605
Epoch 3/25
110288/110288 [==============================] - 8s 69us/step - loss: 2.2069 - acc: 0.5470 - val_loss: nan - val_acc: 0.6232
Epoch 4/25
110288/110288 [==============================] - 8s 69us/step - loss: 1.6555 - acc: 0.6489 - val_loss: nan - val_acc: 0.6668
Epoch 5/25
110288/110288 [==============================] - 8s 69us/step - loss: 1.3931 - acc: 0.6761 - val_loss: nan - val_acc: 0.6821
Epoch 6/25
110288/110288 [==============================] - 8s 68us/step - loss: 1.2370 - acc: 0.6862 - val_loss: nan - val_acc: 0.6937
Epoch 7/25
110288/110288 [==============================] - 8s 69us/step - loss: 1.1204 - acc: 0.7065 - val_loss: nan - val_acc: 0.72

## Analysis of Results

A similar analysis to the previous one is performed again with this model:

1. Learning rate = 0.001 - Accuracy: 82% after 30 epochs, as with only 10, the network had low accuracy but it was still steadly learning, so it was extended. Translation after that does not seem as good as the others "new jersey est parfois calme en l' et il est est en en" falling again into the repetition of the most frequent words, probably just an artifact of this example, given the accuracy, or maybe due to not properly separating testing from validation. 
2. Learning rate = 0.01 - Accuracy: 84%. Translation is pretty good although with some 'hesitation' on part of the model, repeating some common words. "new jersey est parfois calme en cours **et et il la en en** avril". Accuracy seems to have stabilized after 10 epochs
3. Learning rate = 0.1 - Accuracy: 83%. Translation seems a little bit worse than the previous one, although the accuracy is similar, it may be an isolated case, or an overfitting of problems to find the local minima,  ("new jersey est parfois calme en l' et il est la neigeux en"). Accuracy seems to stabilize after half the 10 epochs, not learning much afterwards

In general terms, this is a much better model, where the embedding helps the model to find relationships much faster than the previous one. Again, the version of 0.01 learning rate has been selected for best results and example of translation.

### Update after review
Updating the parameters to the suggestion from the reviewer (lower the learning rate at the same time than incresing the GRU units from output_sequence_length to 256 in this case, as it seems that 128 makes the model to fall into an accuracy trap where loss is no longer diminished after a certain point) improved the results to an **accuracy of 84% (barely an improvement in this case, as the embedding seems to do a really good job with very few units) with a translation:** *new jersey est parfois calme en l' automne et il est neigeux en avril* that is a really good one.

In this case, a learning rate of 0.005 reached the 84% in the suggested 10 epochs, but in order to reach that value with a slower learning rate, 25 epochs were needed to get basically the same result and a worse example translation.

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [17]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implementation similar to the original model, just using the bidirectional layer
    
    # Build the layers
    inputs = Input(shape=input_shape[1:]) # Needs to return an iterable, even if dim 1
    # We need the full output of the hidden layer to pass along
    bd = Bidirectional(GRU(128, activation='tanh', return_sequences=True), merge_mode='concat')(inputs)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(bd)
    
    # define a learning rate (hyperparameter)
    learning_rate = 0.005

    # Build the model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_bd_model(bd_model)

# Train and Print prediction(s)
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_bd_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_bd_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print("Translate '{}': \n".format(english_sentences[0]))
print(logits_to_text(simple_bd_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 13s 115us/step - loss: 1.7740 - acc: 0.5881 - val_loss: nan - val_acc: 0.6473
Epoch 2/10
110288/110288 [==============================] - 12s 104us/step - loss: 1.1460 - acc: 0.6667 - val_loss: nan - val_acc: 0.6818
Epoch 3/10
110288/110288 [==============================] - 12s 104us/step - loss: 1.0042 - acc: 0.6885 - val_loss: nan - val_acc: 0.6960
Epoch 4/10
110288/110288 [==============================] - 12s 104us/step - loss: 0.9206 - acc: 0.7016 - val_loss: nan - val_acc: 0.7066
Epoch 5/10
110288/110288 [==============================] - 11s 103us/step - loss: 0.8699 - acc: 0.7104 - val_loss: nan - val_acc: 0.7148
Epoch 6/10
110288/110288 [==============================] - 11s 103us/step - loss: 0.8369 - acc: 0.7165 - val_loss: nan - val_acc: 0.7053
Epoch 7/10
110288/110288 [==============================] - 12s 105us/step - loss: 0.8055 - acc: 0.7241 - val_loss: nan -

## Analysis of Results

A similar analysis to the previous ones is performed again with the bidirectional model:

1. Learning rate = 0.001 - Accuracy: 67% after 50 epochs, as with only 10, steady growth till epoch 10, with 60% accuracy, but after that it flattens around 65 in about 5 more epochs, very little progress from there. The translation is not good *"new jersey est parfois calme en l' et il est est en en"* again, falling into the repetition trap.
2. Learning rate = 0.01 - Accuracy: 67%. with 65% acc reached by epoch 5 and flattening from then on. Translation is about the same quality as above, with same caveats. *"new jersey est parfois calme en mois et il est est en en"*. 
3. Learning rate = 0.1 - Accuracy: 64%. Translation seems a little bit worse than the previous one, with slightly worse accuracy too,  *"new jersey est parfois calme en en et il est est en en "*. Accuracy seems to stabilize after 3-4 epochs, but lack precision, not learning much afterwards

In general terms, this is a much better model than the first one, but not as good as the second. Still, it shows there are other techniques that we could apply to the basic RNN that improves its capabilities. Only thing left would be check if by combining both techniques, we can take advantage of both, seeing the past and the future, and the abstraction of embedding to facilitate mathematical operations.

### Update after review
Updating the parameters to the suggestion from the reviewer (lower the learning rate at the same time than incresing the GRU units from output_sequence_length to 128) improved the results to an **accuracy of 75% with a translation:** *new jersey est parfois calme au mois de il il il chaud en en*. Quite an improvement.

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [ ]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    return None
tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [11]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement, by following a similar process as before
    # Build the layers
    inputs = Input(shape=input_shape[1:])
    embeds = Embedding(input_dim=english_vocab_size,
                       output_dim=128)(inputs)
    # The rest of the model is similar, just we operate with the embeddings now
    hidden_bd = Bidirectional(GRU(128, activation='tanh',
                                  return_sequences=False,
                                  dropout = 0.3, recurrent_dropout=0.3))(embeds)
    hidden_bd = RepeatVector(output_sequence_length)(hidden_bd)
    hidden_bd = Bidirectional(GRU(256,
                                  activation='tanh',
                                  return_sequences=True,
                                  dropout = 0.3, recurrent_dropout=0.3))(hidden_bd)
    hidden_bd = TimeDistributed(Dense(2 * french_vocab_size, activation='relu'))(hidden_bd)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(hidden_bd)
    
    # define a learning rate (hyperparameter)
    learning_rate = 0.005
    
    # Build the model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    model.summary()
    
    return model

tests.test_model_final(model_final)
    
print('Final Model Loaded')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           25472     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               197376    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 21, 512)           787968    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 688)           352944    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 344)           237016    
Total para

## Prediction (IMPLEMENTATION)

In [12]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # Train neural network using model_final
    
    # Pad the input 
    x = pad(x, y.shape[1])
    
    # Build the model
    model = model_final(x.shape, y.shape[1], len(x_tk.word_counts)+1, len(y_tk.word_counts)+1)
    
    # Fit the model with the data
    model.fit(x, y, batch_size=1024, epochs=35, validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 21, 128)           25600     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               197376    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 21, 512)           787968    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 21, 690)           353970    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 345)           238395    
Total para

## Analysis of Results

In this case, the approach to the network architecture is slightly different. 

In a first shot, a try was done incorporating both the Bidirectional and the embeddings, with basically the same parameters as the previous networks. This was done because until this point, the intent behind using the same basic architecture was to be able to compare results using the same ideas.

The resultant networks allowed to assess both the impact of the basic idea in a simple way, but this was it, it was a simple network as reflected by the summary of the model (about 15 to 30 thousand different parameters). So when combining all the different ideas, the number of parameters that the network was able to incorporate was not able to capture the complexity of the problem. This resulted in several effects:

- Slow training: Suddenly, the 10 epochs even with the standard learning rate of 0.1 provided a steady but slower learning than in previous architectures.
- Worse results than the simpler solutions: Even after adding epochs (as much as 100) the network topped at 83% validation accuracy. It was still growing when cut off, but it came to a crawl and would have needed at least 500 in order to improve 3-4% and be closer to 90% desired. This result is worse than the embeddings, with a much lower requirement in computational power.

So there was the need to modify the model in order to accomodate the added complexity provided by both techniques. 

A first attemp previous to introducing additional complexity was using dropout. By normalizing the network and allowing some "forgetfulness", it was expected than the last epochs, where the variation of loss and validation was very small, improved, and it was indeed, but the gains very very modest (from 82% to 84% approx.). Even more so, this was not consistent and there seemed to be a sweet spot where the dropout value gave the biggest improvements (dropout = 0.2) but larger or smaller values resulted in no improvement, to slower learning -as it was in part expected- that resulted in same or worse results after 100 epochs.

As this was not enough and there were still some issues with the translations *"new jersey est parfois chaud pendant l' automne et il est neigeux en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"* (missing the last word) a new dense layer was introduced before the last one that allowed the model to abstract a larger number of concepts, changing the number of parameters from 30 thousand to almost half a million.
    
But training it was a different issue, as the network learned really fast, enough to go above 90% validity accuracy after only 30 epochs, but with some glitches:

- Validity accuracy (and loss) was always around 4-5% better (higher for accuracy, lower for loss) than training, consistently, over all the epochs. This was unexpected and normally the problem is the inverse where there is overtraining.
- After certain point, the system became "amnesiac", for a lack of a better word to describe it, as it showed an increased loss with every step at certain point that only stopped once a loss of about 3 or 4 was established. The best guess is that the gradient changed signs making the descent become "ascent" until it corrected itself. After that point, recovery was not reached, or not at the same pace. The solution has been to cut the fitting before this state was reached, but letting it go further, it consistently reached the amnesiac state with the network shown, after about 40-50 epochs.

As a summary, including new ideas has consistently improved the model and the results, although it required the addition of new layers or techniques and more care in the training before reaching satisfactory results, and encorauges to learn more about the issues encountered during the exercise.

### Update after review
By adopting the suggestions from the reviewer, the accuracy reached the desired levels and the issue present related to the sudden loss of accuracy didn't present again, obtaining **accuracy of 97.7% with an accurate translation for both sentences**. The greater gains are as usual located in the first epochs (7-8) and after than 95% is obtained after 10 epochs. Another 25 barely added a couple of percentage points, but these are the ones that manage to get accurate translations almost all the time compared to having always one or two words off.

It is interesting to notice that the loss and accuracy for both training and validation sets are more in accordance now, compared to the previous scenario causing the issue described above.

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [13]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 360812 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?